In [1]:
import pandas as pd
import numpy as np
import os

from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [2]:
source = 'source/sbp_bmi_tc_fpg_agestandardized_12911.xls'

In [18]:
import xlrd

In [20]:
xls = xlrd.open_workbook(source, on_demand=True)
print(xls.sheet_names())

['SBP mean (mm Hg)', 'BMI mean (kg per m2)', 'TC mean (mmol per L)', 'FPG mean (mmol per L)', 'Diabetes prevalence']


In [5]:
data = pd.read_excel(source)

In [6]:
data.head()

,Country,Year,Female,Unnamed: 3,Unnamed: 4,Unnamed: 5,Male,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Age-standardized mean,Posterior standard deviation,Lower 95% uncertainty interval,Upper 95% uncertainty interval,Age-standardized mean,Posterior standard deviation,Lower 95% uncertainty interval,Upper 95% uncertainty interval
1,Afghanistan,1980.0,122.08,5.75904,110.607,133.52,125.199,5.56107,114.289,136.468
2,Afghanistan,1981.0,122.268,5.45042,111.393,133.135,125.216,5.17833,115.225,135.713
3,Afghanistan,1982.0,122.487,5.18192,112.132,132.857,125.249,4.84846,115.819,134.997
4,Afghanistan,1983.0,122.706,4.95718,112.872,132.638,125.309,4.5781,116.498,134.653


In [7]:
data = data[['Country', 'Year', 'Female', 'Male']].drop(0)

In [8]:
data.head()

,Country,Year,Female,Male
1,Afghanistan,1980.0,122.08,125.199
2,Afghanistan,1981.0,122.268,125.216
3,Afghanistan,1982.0,122.487,125.249
4,Afghanistan,1983.0,122.706,125.309
5,Afghanistan,1984.0,122.909,125.344


In [21]:
# get all data

all_data = {}

for i in xls.sheet_names():
    df = pd.read_excel(source, sheetname=i)
    df = df[['Country', 'Year', 'Female', 'Male']].drop(0)
    
    all_data[i] = df

In [9]:
# country

In [11]:
country = data[['Country']].drop_duplicates().copy()

In [13]:
country['country'] = country['Country'].map(to_concept_id)

In [14]:
country.columns = ['name', 'country']

In [16]:
country.to_csv('../ddf--entities--country.csv', index=False)

In [29]:
# sex

sex = pd.DataFrame([['male', 'Male'], ['female', 'Female']], columns=['sex', 'name'])

In [30]:
sex.to_csv('../ddf--entities--sex.csv', index=False)

In [17]:
# concepts

In [23]:
all_concepts = [x for x in xls.sheet_names()]
all_concept_ids = [to_concept_id(x) for x in all_concepts]

In [32]:
concepts = pd.DataFrame([], columns=['concept', 'name', 'concept_type'])

In [33]:
concepts['concept'] = ['name', 'year', 'sex', 'country', *all_concept_ids]
concepts['name'] = ['Name', 'Year', 'Sex', 'Country', *all_concepts]

In [34]:
concepts['concept_type'] = 'measure'

In [36]:
concepts.iloc[0]['concept_type'] = 'string'
concepts.iloc[1]['concept_type'] = 'time'
concepts.iloc[2]['concept_type'] = 'entity_domain'
concepts.iloc[3]['concept_type'] = 'entity_domain'

In [38]:
concepts.to_csv('../ddf--concepts.csv', index=False)

In [ ]:
# datapoints

In [39]:
data.columns = list(map(to_concept_id, data.columns))

In [45]:
data['country'] = data['country'].map(to_concept_id)

In [46]:
data_ = data.set_index(['country', 'year'])

In [48]:
data_.stack().reset_index().head()

,country,year,level_2,0
0,afghanistan,1980.0,female,122.08
1,afghanistan,1980.0,male,125.199
2,afghanistan,1981.0,female,122.268
3,afghanistan,1981.0,male,125.216
4,afghanistan,1982.0,female,122.487


In [ ]:
for k, df in all_data.items():
    df.columns = list(map(to_concept_id, df.columns))
    df['country'] = df['country'].map(to_concept_id)
    
    df = df.set_index(['country', 'year'])
    
    df = data_.stack().reset_index()
    
    df.columns = ['country', 'year', 'sex', k]
    
    path = '../ddf--datapoints--{}--by--country--sex--year.csv'.format(k)
    df.to_csv(path, index=False)